In [ ]:
# Importation
import numpy as np
import pandas as pd
import plotly.express as px

# **I - Chargement des données**

In [2]:
# Chargement des données
df = pd.read_csv("../data/01_251021_Notations200.csv")

# Ajout de nouvelles colonnes
### Calcul du score humain moyen
df["score_humain"] = df[["Garance", "Matthias", "Yannis"]].mean(axis=1)

### Nombre de charactères par contribution et par extractions
df["nb_char_contrib"] = df["contribution"].apply(len)
df["nb_char_extraction"] = df["ideas_text"].apply(len)

### Hallucinations et Idées_non_ind en catégories
df['hal_cat'] = df['Hallucinations'].astype("category")
df["idees_cat"] = df["Idées_non_ind"].astype("category")

### Catégories de longueur
length_bins = [0, 250, 500, 2000, float("inf")]
length_labels = ["Très courte (0-250)", "Courte (251-500)", "Moyenne (501-2000)", "Longue (2000+)"]
df["contrib_cat"] = pd.cut(df["nb_char_contrib"], bins=length_bins, labels=length_labels)

# **II - Analyse de la métrique**

In [3]:
# Corrélation de Pearson entre score humain et score QualIT
correlation = df["score_humain"].corr(df["C"])
print(f"Corrélation entre le score humain moyen et le score QualIT : {correlation:.2f}")

Corrélation entre le score humain moyen et le score QualIT : 0.71


La corrélation entre le score humain moyen et la métrique QualIT est assez élevée ***ET*** positive, ce qui suggère qu'elle n'est pas trop mauvaise pour évaluer la qualité de l'extraction. Toutefois, ce résultat doit être interprété avec prudence pour 2 raisons :
- Le désaccord entre évaluateurs humains indique que la notation n'est pas totalement fiable.
- La métrique ne tient pas compte de la pertinence des idées extraites, seulement de leurs similarités avec la contribution originale.

In [4]:
fig = px.scatter(
    df, x="C", y="score_humain", 
    color="hal_cat", color_discrete_sequence=["#ff6361", "#003f5c"]
)
# Ajout de la ligne x = y pour référence
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title={"text": "<b>Comparaison du score humain avec la métrique QualIT</b>"},
    xaxis_title="Métrique QualIT",
    yaxis_title="Score humain",
    width=700, height=500
)
fig.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
fig.show()

In [5]:
fig = px.scatter(
    df, x="C", y="score_humain", 
    color="idees_cat", color_discrete_sequence=["#ff6361", "#003f5c"]
)
# Ajout de la ligne x = y pour référence
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title={"text": "<b>Comparaison du score humain avec la métrique QualIT</b>"},
    xaxis_title="Métrique QualIT",
    yaxis_title="Score humain",
    width=700, height=500
)
fig.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
fig.show()

In [6]:
df["log_score_humain"] = np.log10(df["score_humain"]+1)

fig = px.scatter(
    df, x="C", y="log_score_humain", 
    color="idees_cat", color_discrete_sequence=["#ff6361", "#003f5c"]
)
# Ajout de la ligne x = y pour référence
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=1, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title={"text": "<b>Comparaison du score humain avec la métrique QualIT</b>"},
    xaxis_title="Métrique QualIT",
    yaxis_title="Score humain",
    width=700, height=500
)
fig.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
fig.show()

Les extractions échouées ne sont pas sur les graphiques précédents car elles n'ont pas de valeur dans "Hallucinations" ou "Idées_non_ind".

In [7]:
fig = px.scatter(
    df, x="C", y="score_humain", 
    color="contrib_cat", color_discrete_sequence=["#ffd380", "#ff6361", "#8a508f", "#00202e"]
)
# Ajout de la ligne x = y pour référence
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title={"text": "<b>Comparaison du score humain avec la métrique QualIT</b>"},
    xaxis_title="Métrique QualIT",
    yaxis_title="Score humain",
    width=700, height=500
)
fig.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
fig.show()

In [8]:
fig = px.histogram(
    df, x="nb_char_contrib", y="C", nbins=max(df["nb_char_contrib"])//100,
    color_discrete_sequence=["#2a2781"], histfunc="avg"
)
fig.update_layout(
    title={"text": "<b>Métrique QualIT en fonction du nombre de charactères par contribution<b>"}, 
    xaxis_title="Nombre de charactères", 
    yaxis_title="Moyenne QualIT", 
    width=800, height=450
)
fig.update_traces(
    hovertemplate =
        "Nombre de charactères = %{x}<br>" \
        "Moyenne QualIT = %{y}<extra></extra>"
)
fig.show()

In [9]:
fig = px.histogram(
    df, x="nb_char_extraction", y="C", nbins=max(df["nb_char_extraction"])//50,
    color_discrete_sequence=["#2a2781"], histfunc="avg"
)
fig.update_layout(
    title={"text": "<b>Métrique QualIT en fonction du nombre de charactères par extraction<b>"}, 
    xaxis_title="Nombre de charactères", 
    yaxis_title="Moyenne QualIT", 
    width=800, height=450
)
fig.update_traces(
    hovertemplate =
        "Nombre de charactères = %{x}<br>" \
        "Moyenne QualIT = %{y}<extra></extra>"
)
fig.show()